In [1]:
# Library Call
import re
import nltk
import html
import csv
import numpy as np

In [2]:
# All Cleaning functions

def init(text):
# from html.parser import HTMLParser

# html_parser = HTMLParser()
    original_tweet = text
    text = html.unescape(original_tweet)
    text = text.replace("\\n", ". ")    # Removed new line character from input data
#     print ("\n0. ", text)
    return text

## Decoding Data
# tweet = original_tweet.decode("utf8").encode('ascii', 'ignore')


def removeUnnecessaryWords(text):
    from nltk.tokenize import word_tokenize
    words = word_tokenize(text)

    appostophes = {"'s" : "is", "'re" : "are", "'ve" : "have", "'d" : "would",
                   "'m" : "am", "'t" : "not", "'ll" : "shall", "ca" : "can", "n't" : "not"} 
        
    # More to the list must be added

    ignore_words = ['?', '-', '.', '_', '/', ' ', '.', '!', '\n', '#'] #,
#                "you'll", 'itself', 'some', 'same', 'off', 'any', 'having',
#                 'and', 'theirs', 'your', 'should', 'after', 'out', 'in', 
#                 "you'd", 'd', 'its', 'had', 'myself','from', 'ourselves', 
#                 'here', 'an', 'all', 'yours', 'as', 'hers', 'they', 'll',
#                 "she's", 'through', 'you', 'then', 'once', 'my', 'am', 'who',
#                 'being', 'of', 'shan', 'that', 'so', 'with', 'yourselves',
#                 'both', 't', 'his', 'we', 'more', 'did', 'our', 'he', 'o', 
#                 'them', 'than', 'it', 'y', 'her', 'up', 'about', 'this', 
#                 'himself', 'just', 'if', 'own', 'has', 'how', 'because', 
#                 'him', 'doing', 'at', 'm', 'is', 'each', 's', 'too', 'those', 
#                 'such', 'have', 'above', "you've", 'most', 'on', 'under', 
#                 'by', 'few', 'where', 'when', 'were', "you're", "it's",
#                 'been', 'the', 'before', 'do', 'these', 'other', 'to', 'i',
#                 'can', 'themselves', 'what', 'are', 'while', 'which', 'me',
#                 'ma', "that'll", 've', 'for', 'why', 'a', 'during', 'yourself',
#                 'below', 'now', 'only', 'their', 'herself', 'will', 'does', 
#                 'she', 'be', 'there', "should've", 'was', 're', 'ours', 
#                 'whom', 'further',' ']

    # Appostophes word removal
    reformed = [appostophes[word] if word in appostophes else word for word in words]
#     print ("1.1. ", reformed, "\n")
    
    
    
    # Hashtags and @References Removed
    i = 0
    while(i < len(reformed)):
        word = reformed[i]
#         if word == "#" or word == "@":
        if word == "@":
            if i+1 < len(reformed):
                del reformed[i+1]
            del reformed[i]
            i -= 1
        i += 1
            
#     print("1.2. ", reformed, "\n")
    
    # Turn all caps words (E.g: 'WHAT') to lowercases
    reformed2 = []
    for i, word in enumerate(reformed):
        if re.match('[A-Z]+', word):
            reformed2 = reformed[:i]
            reformed2.append(reformed[i].lower())
            reformed = reformed2 + reformed[i+1:]
#     print("1.3. ", reformed, "\n")
    
    # Stop words and punctuation removal
    reformed2 = []
    for i, word in enumerate(reformed):
        if word.lower() not in ignore_words:
            reformed2.append(word)
    reformed = " ".join(reformed2)
#     print ("1.4. ", reformed, "\n") 
    return reformed



def splitAttachedWords(text):
    # Fixing the lower cased front removed error, by changing the 1st 'letter' to uppercase
    for i, c in enumerate(text):
        if (c >= 'a' and c <= 'z') or (c >= 'A' and c <= 'Z'):
            text = text[:i] + text[i].upper() + text[i+1:]
            break
    
    cleaned = " ".join(re.findall('[A-Z][^A-Z]*', text))
#     print ("2. ", cleaned, "\n")
    return cleaned



def standardizeWords(text):
    cleaned = text
    import itertools
    cleaned = ''.join(''.join(s)[:2] for _, s in itertools.groupby(cleaned))
#     print ("3. ", cleaned, "\n")
    return cleaned



def removeURLs(text):
    cleaned = text
    url_reg  = r'[a-z]*[:.]+\S+'
    result   = re.sub(url_reg, '', cleaned)
#     print ("4. ", result, "\n")
    return result

In [3]:
# Single function defining the whole cleaning process
def clean_tweet(text):
    text = init(text)
    text = removeUnnecessaryWords(text)
    text = splitAttachedWords(text)
    text = standardizeWords(text)
    text = removeURLs(text)
    return text

In [4]:
def clean_file(inpFile, outFile):
    # Initializing
    tweets = []
    anger = []
    anticipation = []
    disgust = []
    fear = []
    joy = []
    love = []
    optimism = []
    pessimism = []
    sadness = []
    surprise = []
    trust = []


    # Reading file
    with open(inpFile) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            tweets.append(row['Tweet'])
            anger.append(row['anger'])
            anticipation.append(row['anticipation'])
            disgust.append(row['disgust'])
            fear.append(row['fear'])
            joy.append(row['joy'])
            love.append(row['love'])
            optimism.append(row['optimism'])
            pessimism.append(row['pessimism'])
            sadness.append(row['sadness'])
            surprise.append(row['surprise'])
            trust.append(row['trust'])

    # Cleaning the tweets
    cleaned_tweets = []
    for line in tweets:
        cleaned_tweets.append(clean_tweet(line))


    # Building ouput
    out = []
    for i, _ in enumerate(cleaned_tweets):
        out.append([cleaned_tweets[i], anger[i], anticipation[i], disgust[i], fear[i], joy[i], love[i], 
                        optimism[i], pessimism[i], sadness[i], surprise[i], trust[i]])


    # out = np.array([tweets, anger, anticipation, disgust, fear, joy, love, 
    #                     optimism, pessimism, sadness, surprise, trust])
    # out.transpose() 
    #
    # I am sure there is a way to avoide the array above using transpose of a matrix, but... I will pass

    outputFile = open(outFile, 'w')
    with outputFile:
        writer = csv.writer(outputFile)
        writer.writerows([['Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 
                           'optimism', 'pessimism', 'sadness', 'surprise', 'trust']])
        writer.writerows(out)

    print("Writing complete")

    outputFile.close();

In [5]:
clean_file('SemEval_Train.csv', 'train.csv')
clean_file('SemEval_Dev.csv', 'test.csv')

Writing complete
Writing complete
